# DAT600


### Ardijan Rexhaj, Daniel Alvsåker, Ove Oftedal


## Task 1:

### a\) 1.

1 -> [2] <br>
2 -> [2, 3, 4] <br>
3 -> [1, 2, 5] <br>
4 -> [5, 6] <br>
5 -> [3, 4] <br>
6 -> [4] <br>


### a\) 2.

```
     1◄───┐
     │    │
     ▼    │
  ┌─►2◄──►3
  └──┤    ▲
     │    │
     ▼    ▼
     4◄──►5
     ▲
     │
     ▼
     6
```

### a\) 3.

A -> [B]
B -> [C,D]
C -> [E,F]
D -> [E,F]
E -> [F,J]
F -> [B,J,G,H]
G -> []
H -> [I]
I -> []
J -> [I]



### b\)

| ![Image 1](./task_b/1.png) | ![Image 2](./task_b/2.png) | ![Image 3](./task_b/3.png) |
|:---:|:---:|:---:|
| ![Image 4](./task_b/4.png) | ![Image 5](./task_b/5.png) | ![Image 6](./task_b/6.png) |
| ![Image 7](./task_b/7.png) | ![Image 8](./task_b/8.png) | ![Image 9](./task_b/9.png) |
| ![Image 10](./task_b/10.png) | ![Image 11](./task_b/11.png) |  |




